In [30]:
!pip install recordlinkage
import recordlinkage
import pandas as pd

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [31]:
df = pd.read_csv("/work/Datos para trabajo de deduplicación.csv", sep=";")
df.set_index('Id')
df.head()

,Id,Nombre,Dirección,EmailAddress,Birthday,Occupation,Kilograms,Centimeters
0,1,Milenka E López,Avda. de la Estación 88,MilenkaLopezCarrillo@dayrep.com,12/20/1979,Home appliance repairer,"71,1",159
1,2,Freya A Cervantes,Plazuela do Porto 87,FreyaCervantesSanabria@superrito.com,02/03/1974,Group exercise instructor,"64,9",166
2,3,Delfor M Ayala,Camiño Ancho 60,DelforAyalaReyes@gustr.com,11/04/1960,Brokerage sales assistant,"93,3",180
3,4,Lisa E Núñez,Fuente del Gallo 43,LisaNunezAlva@dayrep.com,11/07/1966,Parking lot attendant,"88,9",169
4,5,Yakue S Cavazos,Estrela 66,YakueCavazosGuerrero@dayrep.com,02/04/1978,Interpreter,"104,3",175


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            12000 non-null  int64 
 1   Nombre        12000 non-null  object
 2   Dirección     12000 non-null  object
 3   EmailAddress  12000 non-null  object
 4   Birthday      12000 non-null  object
 5   Occupation    12000 non-null  object
 6   Kilograms     12000 non-null  object
 7   Centimeters   12000 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 750.1+ KB


In [35]:
indexer = recordlinkage.Index()  #iniciamos la indexación de los candidatos

indexer.full()  #iniciamos full index con todas las combinaciones posibles de pares de registros
candidate_links = indexer.index(df)

print ("Cantidad de registros del dataset:",len(df), 
"\nCantidad de candidatos:",len(candidate_links))


Cantidad de registros del dataset: 12000 
Cantidad de candidatos: 71994000


In [36]:
# Armamos bloques de candidatos para una variable únicamente 
indexer = recordlinkage.Index()
indexer.block('Nombre')
candidate_links = indexer.index(df)

print ("Cantidad de candidatos:",len(candidate_links))

Cantidad de candidatos: 1185


In [37]:
# Iniciamos las comparaciones usando los métodos de recordlinkage.Compare

compare_cl = recordlinkage.Compare()
compare_cl.string('Nombre', 'Nombre', method='jarowinkler', threshold=0.85, label='Nombre')  #esta es una ventaja de python sobre fril, en python podemos asignar un umbral por campo o por variable, mientras que en fril debemos asignar el umbral para todos los registros de la fila
compare_cl.exact('Birthday', 'Birthday', label='Birthday')
compare_cl.string('EmailAddress', 'EmailAddress', method='damerau_levenshtein', threshold=0.75, label='EmailAddress')
compare_cl.numeric('Centimeters', 'Centimeters')
compare_cl.exact('Kilograms', 'Kilograms')
compare_cl.string('Dirección', 'Dirección', method='jarowinkler', threshold=0.75, label='Dirección')
compare_cl.string('Occupation', 'Occupation', method='jarowinkler', threshold=0.85, label='Occupation')

<Compare>

In [38]:
features = compare_cl.compute(candidate_links, df)
features.head()

,,Nombre,Birthday,EmailAddress,3,4,Dirección,Occupation
9000,0,1.0,1,1.0,1.0,1,1.0,1.0
9003,3,1.0,1,1.0,1.0,1,1.0,1.0
9006,6,1.0,1,1.0,1.0,1,1.0,1.0
9010,10,1.0,1,1.0,1.0,1,1.0,1.0
9012,12,1.0,1,1.0,1.0,1,1.0,1.0


In [39]:
# Ahora debo decidir con cuáles me quedo
features.sum(axis=1).value_counts().sort_index(ascending=False)

7.0    1180
6.0       5
dtype: int64

Casi todos nuestros candidatos son duplicados, por lo que nuestro dataset resultante de duplicados será el mismo que el de los candidatos si especificamos que muestre los que tengan más de 5 coincidencias:

In [40]:
matches = features[features.sum(axis=1) > 5]

print(len(matches))
matches.head()

1185


,,Nombre,Birthday,EmailAddress,3,4,Dirección,Occupation
9000,0,1.0,1,1.0,1.0,1,1.0,1.0
9003,3,1.0,1,1.0,1.0,1,1.0,1.0
9006,6,1.0,1,1.0,1.0,1,1.0,1.0
9010,10,1.0,1,1.0,1.0,1,1.0,1.0
9012,12,1.0,1,1.0,1.0,1,1.0,1.0


#### Realizaremos otra prueba con otros parámetros para comparar los resultados
---
## Prueba 2


In [27]:
compare_cl = recordlinkage.Compare()
compare_cl.string('Nombre', 'Nombre', method='qgram', threshold=0.75, label='Nombre')  #esta es una ventaja de python sobre fril, en python podemos asignar un umbral por campo o por variable, mientras que en fril debemos asignar el umbral para todos los registros de la fila
compare_cl.exact('Birthday', 'Birthday', label='Birthday')
compare_cl.string('EmailAddress', 'EmailAddress', method='levenshtein', threshold=0.85, label='EmailAddress')
compare_cl.numeric('Centimeters', 'Centimeters')
compare_cl.exact('Kilograms', 'Kilograms')
compare_cl.string('Dirección', 'Dirección', method='jaro', threshold=0.80, label='Dirección')
compare_cl.string('Occupation', 'Occupation', method='jarowinkler', threshold=0.75, label='Occupation')

<Compare>

In [28]:
features = compare_cl.compute(candidate_links, df)
features.head()

,,Nombre,Birthday,EmailAddress,3,4,Dirección,Occupation
9000,0,1.0,1,1.0,1.0,1,1.0,1.0
9003,3,1.0,1,1.0,1.0,1,1.0,1.0
9006,6,1.0,1,1.0,1.0,1,1.0,1.0
9010,10,1.0,1,1.0,1.0,1,1.0,1.0
9012,12,1.0,1,1.0,1.0,1,1.0,1.0


In [29]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

7.0    1180
6.0       5
dtype: int64

#### Podemos ver que cambiando algunos parámetros, obtenemos los mismos resultados